# Consumer for Performing Pyrometry Analysis
This consumes the hyprespectral camera data streamed from the initial producer and analyzes it to determine temperatures and emissivities. The code is structured off of notebooks at github.com/openmsi/openmsistream_short_course.

## Config

In [19]:
# imports
import pathlib, importlib, logging, datetime, json, platform
from threading import Thread
from openmsitoolbox.logging import OpenMSILogger
from openmsistream import (
    DataFileDownloadDirectory,
    DataFileStreamProcessor,
    MetadataJSONReproducer,
)

In [20]:
# Configure a logger (only needed when running in a Jupyter notebook like this)
logger = OpenMSILogger("OpenMSIConsumers", filelevel=None)
importlib.reload(logging)

<module 'logging' from '/Users/namanparikh/opt/anaconda3/envs/openmsi/lib/python3.9/logging/__init__.py'>

In [21]:
# The name of the topic to consume files from
CONSUMER_TOPIC_NAME = "tutorial_data"

# Path to the root directory of this repo
repo_root_dir = pathlib.Path().resolve().parent

## Consuming to the Local Filesystem

In [22]:
def download_task(download_directory):
    """Run "reconstruct" for a given DataFileDownloadDirectory, and log some messages
    when it gets shut down

    Args:
        download_directory (DataFileDownloadDirectory): the DataFileDownloadDirectory to run
    """
    # This call to "reconstruct" waits until the program is shut down
    (
        n_read,
        n_processed,
        n_complete_files,
        complete_filepaths,
    ) = download_directory.reconstruct()
    download_directory.close()
    msg = f"{n_read} total messages were consumed"
    if len(complete_filepaths) > 0:
        msg += (
            f", {n_processed} messages were successfully processed, and "
            f'{n_complete_files} file{" was" if n_complete_files==1 else "s were"} '
            "successfully reconstructed"
        )
    else:
        msg += f" and {n_processed} messages were successfully processed"
    msg += (
        f". Most recent completed files (up to {download_directory.N_RECENT_FILES}):\n\t"
    )
    msg += "\n\t".join([str(filepath) for filepath in complete_filepaths])
    download_directory.logger.info(msg)

In [23]:
# Paths to the config file and the directory holding the test files
CONFIG_FILE_PATH = repo_root_dir / "streaming" / "config_files" / "confluent_cloud_broker.config"
TEST_RECO_DIR = repo_root_dir / "streaming" / "reconstructed_test_folder"

In [24]:
CONFIG_FILE_PATH

PosixPath('/Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming/config_files/confluent_cloud_broker.config')

In [25]:
TEST_RECO_DIR

PosixPath('/Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming/reconstructed_test_folder')

In [28]:
# Create the DataFileDownloadDirectory
dfdd = DataFileDownloadDirectory(
    TEST_RECO_DIR,
    CONFIG_FILE_PATH,
    CONSUMER_TOPIC_NAME,
    logger=logger,
)
# Start running its "reconstruct" function in a separate thread
download_thread = Thread(
    target=download_task,
    args=(dfdd,),
)
download_thread.start()

[OpenMSIConsumers 2024-07-18 00:12:19] Will reconstruct files from messages in the tutorial_data topic using 2 threads
[OpenMSIConsumers 2024-07-18 00:18:15] 3 total messages were consumed, 3 messages were successfully processed, and 3 files were successfully reconstructed. Most recent completed files (up to 50):
	test_data.txt
	.ipynb_checkpoints/test_data-checkpoint.txt
	test_watchdog.txt


In [29]:
# Manually shut down the download directory (if running from the command line this would
# be like typing "q" in the Terminal window)
dfdd.control_command_queue.put("q")
download_thread.join()